In [1]:
%run "0_workspace_setup.ipynb"

In [2]:
from ipywidgets import widgets
from IPython.display import display
from nhm_helpers.map_template import make_var_map
from nhm_helpers.nhm_hydrofabric import make_hf_map_elements
from nhm_helpers.nhm_output_visualization import retrieve_hru_output_info
from ipywidgets import HBox, VBox, Button
from nhm_helpers.display_controls import *
from nhm_helpers.output_plots import plot_colors
from nhm_helpers.output_plots import (
    var_colors_dict,
    leg_only_dict,
    make_plot_var_for_hrus_in_poi_basin,
    oopla,
)

poi_id_sel = None

## Introduction
This notebook maps the selected HRU output variable's values and displays in a new browser tab. The list of output variables are user-specified (`selected_output_variables` in [notebook 0](.\0_Workspace_setup.ipynb)). A gage id can be selected and two plots created. The first plot will show a time-series of all HRU values in the selected gage's catchment. The second plot will show a time-series of all HRU output variables averaged for the selected gage's catchment. Both plots will be displayed in a new browser tab. Additionally, maps and plots produced are saved for use outside of notebooks as .html files in the `html_maps` and `html_plots` folders in the `notebook_output` folder.

The cell below reads the NHM subdomain model hydrofabric elements for mapping purposes using `make_hf_map_elements()`, retrieves `pywatershed` output file information for mapping and plotting using `retrieve_hru_output_info()`, and writes general NHM subdomain model run and hydrofabric information.

In [3]:
(
    hru_gdf,
    hru_txt,
    hru_cal_level_txt,
    seg_gdf,
    seg_txt,
    nwis_gages_aoi,
    poi_df,
    gages_df,
    gages_txt,
    gages_txt_nb2,
    HW_basins_gdf,
    HW_basins,
) = make_hf_map_elements(
    NHM_dir,
    model_dir,
    GIS_format,
    param_filename,
    control_file_name,
    nwis_gages_file,
    gages_file,
    default_gages_file,
    nhru_params,
    nhru_nmonths_params,
    nwis_gage_nobs_min,
)

# Retrieve pywatershed output file information for mapping and plotting
plot_start_date, plot_end_date, year_list, output_var_list = retrieve_hru_output_info(
    out_dir,
    water_years,
)

con.print(
    f"{workspace_txt}\n",
    f"\n{gages_txt}{seg_txt}{hru_txt}",
    f"\n     {hru_cal_level_txt}",
)

NHM model domain: Walla_Walla, parameter file: myparam.param
Simulation and observation data range: 01/01/2011 - 12/31/2022 (from control.default.bandit).
 
The parameter file contains 8 gages managed by USGS, 44 segments, and 87 HRUs. 
     85 HRUs are within HWs, and 55 are within HW calibrated with streamflow observations.

<font size=4> &#x270D;<font color='green'>**Enter Information:**</font> **Run the cell below. In the resulting drop-down boxes, select an HRU **output variable** and a **year** value to display in the map. You may also select a gage plots (optional). &#x270D;<font color='green'>**</font><br>
<font size = '3'> The default is set to **recharge**, **mean_annual** with no gage selected, and the map will zoom to the model extent. If a gage number is selected here, the map will zoom to that location. Plots will also be rendered for the selected gage. If no gage is selected (default), the first gage listed in the parameter file will be used. All drop-down box selections can be changed and additional maps and plots will be created. Maps will be displayed and exported (HTML files) to `<NHM subdomain model folder>/notebook_output_files/html_maps`. Plots will be displayed and exported (HTML files) to `<NHM subdomain model folder>/notebook_output_files/html_plots`.

In [ ]:
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Button
from IPython.display import display, IFrame, clear_output

# Style settings
style_var = {"description_width": "initial"}
layout = widgets.Layout(width="25%")

# Variable selector
v = widgets.Dropdown(
    options=output_var_list,
    value=output_var_list[8],
    description="Output variable:",
    layout=layout,
    style=style_var,
)

# Year selector
years = year_list.copy() + ["mean_annual"]
yr = widgets.Dropdown(
    options=years,
    value=years[-1],
    description="Time step (year):",
    layout=layout,
    style=style_var,
)

# Gage combobox
v2 = widgets.Combobox(
    options=poi_df.poi_id.tolist(),
    placeholder="(optional) Enter gage id",
    description="Zoom to gage:",
    ensure_option=True,
    disabled=False,
    layout=layout,
    style=style_var,
)

# Checkboxes for plot types
cb_map = widgets.Checkbox(value=False, description="Include Map")
cb_summary = widgets.Checkbox(value=False, description="Include Summary Time Series")
cb_flux = widgets.Checkbox(value=False, description="Include Flux Time S")
plot_checks = HBox([cb_map, cb_summary, cb_flux])

# Generate button
btn_generate = Button(description="Show Plots", button_style="primary")

# Output areas
out_map = widgets.Output()
out_summary = widgets.Output()
out_flux = widgets.Output()

# Button Click and Display
btn_generate.on_click(on_generate_clicked)
# display(VBox([v, yr, v2, plot_checks, btn_generate, out_map, out_summary, out_flux]))

## Make interactive map of selected output variable values in the NHM subdomain
The following cell creates an interactive folium.map that displays an annual values of all HRUs in the NHM subdomain for a selected output variable. Additionally, variable values for each HRU and additional HRU information can be viewed by left-clicking on HRUs. Maps produced are saved for use outside of notebooks as .html files in `./"subdomain model"/notebook_output/html_maps`.

In [13]:
display(VBox([v, yr, v2, plot_checks, btn_generate, out_map, out_summary, out_flux]))

## Create an interactive time-series plot to evaluate an NHM output variable for poi basin.
The following cell creates a plot for the output variable selected in this notebook that shows a cumulative value for the selected poi basin at the annual, monthly, and daily time steps. Individual poi basin HRU contributions can also be compared to the poi basin values. The plot is interactive and can be saved using the widgets in the upper right-hand corner of the plot. Plots produced are saved for use outside of notebooks as .html files in `./"subdomain model"/notebook_output/html_plots`.